SVM + Word2Vec Classifier
Classify Attack Vector by learning from descriptions.
Classes:  network, adjacent network, local, physical.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import spacy
import seaborn as sns

from sklearn import svm
from sklearn.metrics import classification_report

nlp = spacy.load("en_core_web_lg")

In [2]:
data_train = pd.read_csv('cve_train.csv')
data_test = pd.read_csv('cve_test.csv')

description_train = data_train['description']
description_test = data_test['description']
attackvec_train = data_train['attack_vector']
attackvec_test = data_test['attack_vector']

In [4]:
len(description_test.index)

15404

In [3]:
vectors = description_train.apply(lambda row: sum([token.vector for token in nlp(row)]))

In [11]:
description_train = list(vectors)

clf = svm.SVC(C = 0.1, gamma='scale', kernel='rbf')
clf.fit(description_train, attackvec_train)

SVC(C=0.1)

In [12]:
list_desc = []
for i in range(len(description_test)):
    list_desc.append(description_test.iloc[i])

In [13]:
list_desc_vec = []
for desc in list_desc:
    list_desc_vec.append(sum([token.vector for token in nlp(desc)]))

In [14]:
attackvec_pred = []
for desc_vec in list_desc_vec:
    attackvec_pred.append(clf.predict([desc_vec]))

In [15]:
attackvec_true = []
for i in range(len(attackvec_test)):
    attackvec_true.append(attackvec_test.iloc[i])

In [16]:
target_names = ['NETWORK', 'ADJACENT NETWORK', 'LOCAL', 'PHYSICAL']
print(classification_report(attackvec_true, attackvec_pred, target_names=target_names))

C:\Users\Lenovo\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Lenovo\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                  precision    recall  f1-score   support

         NETWORK       0.00      0.00      0.00       338
ADJACENT NETWORK       0.84      0.46      0.60      3625
           LOCAL       0.82      0.98      0.89     11252
        PHYSICAL       0.00      0.00      0.00       189

        accuracy                           0.82     15404
       macro avg       0.42      0.36      0.37     15404
    weighted avg       0.80      0.82      0.79     15404



C:\Users\Lenovo\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
import pickle
pickle.dump(clf, open('svm_model', 'wb'))